# Gestion de la fidélité client – Analyse Churn Télécom

Notebook conçu pour un entretien de consultant data : il présente une démarche structurée pour analyser le churn client, construire un modèle prédictif et proposer des recommandations actionnables.

## Objectifs du notebook
- Comprendre les facteurs expliquant la résiliation des clients (churn).
- Construire des modèles simples et interprétables pour prédire le churn.
- Préparer un support d'oral clair, avec commentaires pédagogiques.

## Plan du notebook
1. Imports & chargement des données
2. Analyse exploratoire rapide orientée churn
3. Préparation des données & feature engineering
4. Modélisation & comparaison de modèles
5. Interprétabilité & importance des variables
6. Conclusion & recommandations métier
7. Fonction de scoring `predict()`


⚙️ **Préparation de l'environnement**\nSi vous exécutez ce notebook sur une instance fraîche (ex. Google Colab), lancez la cellule suivante une fois pour installer les dépendances externes :\n```python
!pip install shap xgboost
```


In [ ]:
# --- Imports & configuration générale
# Objectif : charger les librairies nécessaires à l'EDA, la modélisation et l'interprétabilité.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier
import shap

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
np.random.seed(42)

## 1. Chargement des données
Nous travaillons sur l'échantillon mis à disposition par l'opérateur Télécom. L'objectif est de partir du fichier brut, de contrôler sa qualité et de prendre en main les variables clés.

In [ ]:
# --- Chargement des données
# On lit directement le fichier CSV via son URL (simulation d'un accès data hébergé).

DATA_URL = "https://raw.githubusercontent.com/YanisBoNueve/wavestone_test/refs/heads/main/data/sujet_B_data_client_churn.csv"

df = pd.read_csv(DATA_URL)
print(f"Dimensions du dataset : {df.shape[0]} lignes, {df.shape[1]} colonnes")
df.head()

In [ ]:
# --- Inspection du schéma de données
# L'objectif est de comprendre les types de variables et d'identifier d'éventuelles incohérences.

df.info()

In [ ]:
# --- Profil des valeurs manquantes
# Nous quantifions les données manquantes pour anticiper les traitements d'imputation.

missing_profile = (
    df.isna()
      .sum()
      .reset_index()
      .rename(columns={"index": "variable", 0: "nb_valeurs_manquantes"})
)
missing_profile["taux_manquant"] = missing_profile["nb_valeurs_manquantes"] / len(df)
missing_profile.sort_values("taux_manquant", ascending=False)

## 2. Analyse exploratoire orientée churn
Approche pragmatique : comprendre comment se répartissent les résiliations et quelles variables semblent discriminantes. On se concentre sur les variables socio-contractuelles fréquemment utilisées dans les cas Télécom.

In [ ]:
# --- Préparation de variables d'analyse
# Création d'indicateurs utiles : taux de churn numérique et nombre de services souscrits.

# Cible numérique pour les calculs de taux
churn_flag = df["Churn"].map({"Yes": 1, "No": 0})

# Liste des services considérés
service_categories = {
    "PhoneService": ["Yes"],
    "MultipleLines": ["Yes"],
    "InternetService": ["DSL", "Fiber optic"],
    "OnlineSecurity": ["Yes"],
    "OnlineBackup": ["Yes"],
    "DeviceProtection": ["Yes"],
    "TechSupport": ["Yes"],
    "StreamingTV": ["Yes"],
    "StreamingMovies": ["Yes"],
}

service_counts = pd.DataFrame({
    col: df[col].isin(valid_values).astype(int)
    for col, valid_values in service_categories.items()
})

df["NumServices"] = service_counts.sum(axis=1)
print("Distribution du nombre de services souscrits :")
df["NumServices"].value_counts().sort_index()

In [ ]:
# --- Taux global de churn
# Première photographie : quelle proportion de clients résilient ?

taux_churn = churn_flag.mean()
print(f"Taux global de churn : {taux_churn:.2%}")

sns.countplot(data=df, x="Churn", palette="Blues")
plt.title("Répartition du churn")
plt.show()

In [ ]:
# --- Fonction utilitaire pour analyser le churn par variable

def churn_rate_by_variable(data, column, bins=None, order=None):
    temp = data.copy()
    temp["ChurnFlag"] = temp["Churn"].map({"Yes": 1, "No": 0})

    group_col = column
    if bins is not None:
        group_col = f"{column}_bin"
        temp[group_col] = pd.cut(temp[column], bins=bins, include_lowest=True)

    summary = (
        temp.groupby(group_col)["ChurnFlag"]
            .agg(["count", "mean"])
            .rename(columns={"count": "nb_clients", "mean": "taux_churn"})
            .reset_index()
    )
    if order is not None:
        summary[group_col] = pd.Categorical(summary[group_col], categories=order, ordered=True)
        summary = summary.sort_values(group_col)

    return summary

In [ ]:
# --- Analyse churn vs Tenure (ancienneté)

tenure_bins = [0, 12, 24, 36, 48, 60, 72]
summary_tenure = churn_rate_by_variable(df, "tenure", bins=tenure_bins)
display(summary_tenure)

sns.barplot(data=summary_tenure, x="tenure_bin", y="taux_churn", palette="viridis")
plt.xticks(rotation=45)
plt.title("Taux de churn par ancienneté (Tenure)")
plt.ylabel("Taux de churn")
plt.xlabel("Ancienneté (mois)")
plt.show()

In [ ]:
# --- Analyse churn vs Contract
summary_contract = churn_rate_by_variable(df, "Contract")
display(summary_contract)

sns.barplot(data=summary_contract, x="Contract", y="taux_churn", palette="viridis")
plt.title("Taux de churn par type de contrat")
plt.ylabel("Taux de churn")
plt.xlabel("Type de contrat")
plt.show()

In [ ]:
# --- Analyse churn vs PaymentMethod
summary_payment = churn_rate_by_variable(df, "PaymentMethod")
display(summary_payment)

sns.barplot(data=summary_payment, x="PaymentMethod", y="taux_churn", palette="viridis")
plt.title("Taux de churn par méthode de paiement")
plt.ylabel("Taux de churn")
plt.xlabel("Méthode de paiement")
plt.xticks(rotation=45, ha="right")
plt.show()

In [ ]:
# --- Analyse churn vs MonthlyCharges
charges_bins = [0, 30, 60, 90, 120]
summary_monthly = churn_rate_by_variable(df, "MonthlyCharges", bins=charges_bins)
display(summary_monthly)

sns.barplot(data=summary_monthly, x="MonthlyCharges_bin", y="taux_churn", palette="viridis")
plt.title("Taux de churn par niveau de facturation mensuelle")
plt.ylabel("Taux de churn")
plt.xlabel("Monthly Charges")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# --- Analyse churn vs PaperlessBilling
summary_paperless = churn_rate_by_variable(df, "PaperlessBilling")
display(summary_paperless)

sns.barplot(data=summary_paperless, x="PaperlessBilling", y="taux_churn", palette="viridis")
plt.title("Taux de churn selon la facturation dématérialisée")
plt.ylabel("Taux de churn")
plt.xlabel("Paperless Billing")
plt.show()

In [ ]:
# --- Analyse churn vs InternetService
summary_internet = churn_rate_by_variable(df, "InternetService")
display(summary_internet)

sns.barplot(data=summary_internet, x="InternetService", y="taux_churn", palette="viridis")
plt.title("Taux de churn selon le type d'accès internet")
plt.ylabel("Taux de churn")
plt.xlabel("Internet Service")
plt.show()

In [ ]:
# --- Relation churn vs nombre de services souscrits
summary_services = churn_rate_by_variable(df, "NumServices")
display(summary_services)

sns.barplot(data=summary_services, x="NumServices", y="taux_churn", palette="viridis")
plt.title("Taux de churn en fonction du nombre de services")
plt.ylabel("Taux de churn")
plt.xlabel("Nombre de services actifs")
plt.show()

## 3. Préparation des données & feature engineering
On prépare maintenant les données pour la modélisation : nettoyage de variables numériques, sélection des features pertinentes et création des objets de prétraitement.

In [ ]:
# --- Copie de travail pour la modélisation
# Objectif : conserver le dataset brut tout en préparant un jeu propre.

df_model = df.copy()

# Conversion de TotalCharges en numérique (certaines valeurs peuvent être vides au départ)
df_model["TotalCharges"] = pd.to_numeric(df_model["TotalCharges"], errors="coerce")

# Vérification du taux de valeurs manquantes après conversion
missing_after_conversion = df_model["TotalCharges"].isna().mean()
print(f"Taux de valeurs manquantes sur TotalCharges après conversion : {missing_after_conversion:.2%}")

# Séparation features / cible
if "customerID" in df_model.columns:
    df_model = df_model.drop(columns=["customerID"])

X = df_model.drop(columns=["Churn"])
y = df_model["Churn"].map({"Yes": 1, "No": 0})

print(f"Nombre de features disponibles pour la modélisation : {X.shape[1]}")

In [ ]:
# --- Identification des types de variables
# Cela permet de construire un pipeline d'encodage adapté.

numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object", "bool"]).columns.tolist()

print("Variables numériques :", numeric_features)
print("Variables catégorielles :", categorical_features)

In [ ]:
# --- Préprocesseurs pour pipeline de modélisation
# Imputation simple + encodage One-Hot pour les catégorielles.

numeric_transformer = SimpleImputer(strategy="median")
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [ ]:
# --- Découpage train/test
# Un split simple (70/30) permet de se positionner rapidement sur la performance des modèles.

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Taille du train : {X_train.shape[0]} lignes")
print(f"Taille du test : {X_test.shape[0]} lignes")

## 4. Modélisation & comparaison
Nous comparons trois modèles complémentaires : régression logistique (baseline linéaire explicable), forêt aléatoire (non-linéaire robuste) et XGBoost (boosting gradient). L'objectif est de choisir un compromis performance / interprétabilité.

In [ ]:
# --- Entraînement des modèles
# Chaque modèle est intégré dans un pipeline partageant le même préprocesseur.

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=300, max_depth=None, random_state=42, class_weight="balanced"),
    "XGBoost": XGBClassifier(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        random_state=42,
        use_label_encoder=False
    ),
}

trained_models = {}
metrics = []
confusion_matrices = {}

for name, model in models.items():
    pipe = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", model)])
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    metrics.append({"Modèle": name, "Accuracy": acc, "F1": f1})
    confusion_matrices[name] = confusion_matrix(y_test, y_pred)

    trained_models[name] = pipe

metrics_df = pd.DataFrame(metrics).sort_values(by="F1", ascending=False)
metrics_df

In [ ]:
# --- Visualisation de la matrice de confusion du meilleur modèle

best_model_name = metrics_df.iloc[0]["Modèle"]
best_pipeline = trained_models[best_model_name]
best_conf_matrix = confusion_matrices[best_model_name]

plt.figure(figsize=(5, 4))
sns.heatmap(best_conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title(f"Matrice de confusion – {best_model_name}")
plt.xlabel("Prédictions")
plt.ylabel("Réalité")
plt.show()

print(f"Meilleur modèle sur F1-score : {best_model_name}")

## 5. Interprétabilité des modèles
Objectif : identifier les facteurs qui influencent le plus la prédiction. On s'appuie sur l'importance des variables des modèles arborescents et un aperçu SHAP.

In [ ]:
# --- Importance des variables pour les modèles arborescents
# Extraction des noms de features après prétraitement pour lecture métier.

def get_feature_names(fitted_preprocessor):
    return fitted_preprocessor.get_feature_names_out()

feature_names = get_feature_names(best_pipeline.named_steps["preprocessor"])

importance_results = {}
for name in ["Random Forest", "XGBoost"]:
    if name in trained_models:
        clf = trained_models[name].named_steps["classifier"]
        if hasattr(clf, "feature_importances_"):
            importances = clf.feature_importances_
            importance_df = (
                pd.DataFrame({"feature": feature_names, "importance": importances})
                  .sort_values("importance", ascending=False)
                  .head(15)
            )
            importance_results[name] = importance_df
            display(f"Top features – {name}")
            display(importance_df)
            plt.figure(figsize=(8, 5))
            sns.barplot(data=importance_df, x="importance", y="feature", palette="mako")
            plt.title(f"Importance des variables ({name})")
            plt.xlabel("Importance normalisée")
            plt.ylabel("Feature")
            plt.show()

In [ ]:
# --- Analyse SHAP pour le meilleur modèle arborescent
# Permet de visualiser l'influence moyenne de chaque feature.

shap_model_name = None
for candidate in ["XGBoost", "Random Forest"]:
    if candidate == best_model_name:
        shap_model_name = candidate
        break
    if shap_model_name is None and candidate in trained_models:
        shap_model_name = candidate

if shap_model_name is not None:
    tree_pipeline = trained_models[shap_model_name]
    tree_clf = tree_pipeline.named_steps["classifier"]
    X_train_transformed = tree_pipeline.named_steps["preprocessor"].transform(X_train)
    if hasattr(X_train_transformed, "toarray"):
        X_train_transformed = X_train_transformed.toarray()

    feature_names_shap = tree_pipeline.named_steps["preprocessor"].get_feature_names_out()

    sample_size = min(1000, X_train_transformed.shape[0])
    sample_indices = np.random.choice(X_train_transformed.shape[0], sample_size, replace=False)
    X_sample = X_train_transformed[sample_indices]

    explainer = shap.TreeExplainer(tree_clf)
    shap_values = explainer.shap_values(X_sample)

    if isinstance(shap_values, list):
        shap_values_to_plot = shap_values[1]
    else:
        shap_values_to_plot = shap_values

    shap.summary_plot(shap_values_to_plot, features=X_sample, feature_names=feature_names_shap, plot_type="bar", show=False)
    plt.title(f"Importance SHAP – {shap_model_name}")
    plt.show()
else:
    print("Aucun modèle arborescent disponible pour SHAP.")

## 6. Conclusion & recommandations
- **Relation nombre de services / churn** : l'analyse montre que les clients multi-équipés résilient moins que ceux avec peu de services. L'hypothèse "plus de services ↔ churn" est globalement infirmée : développer l'upsell multi-services peut renforcer la fidélité.
- **Facteurs clés** : l'ancienneté faible, les contrats sans engagement et la facturation électronique ressortent comme signaux de risque. Les charges mensuelles élevées augmentent légèrement le churn.
- **Modèle retenu** : le meilleur modèle identifié (cf. cellule précédente – observé comme Random Forest sur l'échantillon) offre le meilleur équilibre F1 / interprétabilité. La forêt aléatoire fournit des importances lisibles et une bonne robustesse opérationnelle.

### Prochaines étapes proposées
1. **Validation & tuning léger** : cross-validation 5-fold, ajustement des hyperparamètres clés (profondeur forêt, régularisation XGBoost).
2. **Amélioration qualité data** : suivi des valeurs manquantes `TotalCharges`, enrichissement avec des signaux d'usage réseau et historiques d'interactions SAV.
3. **Industrialisation MLOps** : pipeline automatisé (feature store, modèle registry, suivi du drift et des performances), monitoring des scores en production.
4. **Adoption métier** : guides d'usage pour les équipes fidélisation, alertes sur les clients à risque et intégration dans les parcours CRM.


## 7. Fonction de scoring `predict()`
La fonction ci-dessous illustre comment intégrer le modèle dans un produit data : on lui passe un dictionnaire représentant un client, et elle retourne la probabilité de churn et la classe prédite.

In [ ]:
# --- Fonction utilitaire de scoring
# Elle permet de simuler l'utilisation du meilleur modèle dans un outil métier.

feature_reference = X.columns.tolist()


def predict(client_dict):
    '''Retourne la probabilité de churn et la prédiction binaire pour un client.'''
    client_df = pd.DataFrame([client_dict])

    # Garantir que toutes les features attendues sont présentes
    missing_features = set(feature_reference) - set(client_df.columns)
    for feat in missing_features:
        client_df[feat] = np.nan

    client_df = client_df[feature_reference]

    proba = best_pipeline.predict_proba(client_df)[0, 1]
    prediction = "Yes" if proba >= 0.5 else "No"

    return {"churn_probability": float(proba), "churn_prediction": prediction}


# Exemple d'utilisation (les valeurs sont indicatives)
example_client = {
    "gender": "Female",
    "SeniorCitizen": 0,
    "Partner": "Yes",
    "Dependents": "No",
    "tenure": 5,
    "PhoneService": "Yes",
    "MultipleLines": "No",
    "InternetService": "Fiber optic",
    "OnlineSecurity": "No",
    "OnlineBackup": "No",
    "DeviceProtection": "No",
    "TechSupport": "No",
    "StreamingTV": "Yes",
    "StreamingMovies": "Yes",
    "Contract": "Month-to-month",
    "PaperlessBilling": "Yes",
    "PaymentMethod": "Electronic check",
    "MonthlyCharges": 95.5,
    "TotalCharges": 500.0,
    "NumServices": 5
}

predict(example_client)